In [1]:
import numpy as np
import sympy as sym
from sympy import Function, hessian, pprint, Matrix, simplify
import math
from scipy.spatial.transform import Rotation as R
from sympy.parsing.sympy_parser import parse_expr
import time

sym.init_printing()

def angles_to_rad (q):
    for i in range (q.size):
        q[i] = np.deg2rad(q[i])
        
def print_pose(pose):
    print("position of robot flange:", pose[0:3,3])
    print ("x vector:" , pose[0:3:,0])
    print ("y vector:" , pose[0:3:,1])
    print ("z vector:" , pose[0:3:,2]) 

# 24 nominal parameters of kinematics
def get_nominal_DH_kinematic_parameters():
    kinematic_parameters =                                  np.array([525,150,0,-90], dtype = 'float')
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([0,790,-90,180])))
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([0,250,0,-90])))    
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([-1040,0,0,90])))
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([0,0,0,-90])))
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([-100,0,0,0])))
    return kinematic_parameters

print("symbolic calculations started",time.strftime("%H:%M:%S", time.localtime()))
def generate_DH_error_symbols(n):
    generated_symbols = []
    for i in range (n):
        generated_symbols.append(sym.symbols('δd'+ str(i)))
        generated_symbols.append(sym.symbols('δa'+ str(i)))
        generated_symbols.append(sym.symbols('δθ'+ str(i)))
        generated_symbols.append(sym.symbols('δα'+ str(i)))
    return generated_symbols

d, a,θ,α = sym.symbols('d a θ α')
q_symbols = sym.symbols('q:6')
error_symbols = generate_DH_error_symbols(6)   

def get_symbolic_matrix_with_error (δd,δa,δθ,δα, kinematics_params, q):
    [d0,a0,θ0,α0] = kinematics_params
    θ0 = np.deg2rad(θ0)
    α0 = np.deg2rad(α0)
    d = d0 + δd
    a = a0 +δa
    θ = θ0 + q + δθ
    α = α0 + δα
    
    dh = sym.eye(4)
    dh[0,0] = sym.cos(θ)
    dh[0,1] = -sym.sin(θ) * sym.cos(α)
    dh[0,2] = sym.sin(θ) * sym.sin(α)
    dh[0,3] = a * sym.cos(θ)
    dh[1,0] = sym.sin(θ)
    dh[1,1] = sym.cos(θ)*sym.cos(α)
    dh[1,2] = -sym.cos(θ)* sym.sin(α)
    dh[1,3] = a *sym.sin(θ)
    dh[2,1] = sym.sin(α)
    dh[2,2] = sym.cos(α)
    dh[2,3] = d
    return dh

def get_symbolic_kinematics_matrix():
    nominal_kinematics = get_nominal_DH_kinematic_parameters()
    result = sym.eye(4)
    for i in range(6):
        result = result @ get_symbolic_matrix_with_error(
            *error_symbols[4*i:4*(i+1)],nominal_kinematics[i], q_symbols[i])
    return result

symbolic_matrix_with_error = get_symbolic_kinematics_matrix()
get_pose_from_radians = sym.lambdify([q_symbols,error_symbols], 
                                           symbolic_matrix_with_error, "numpy") 

def get_symbolic_jacobian():
    position_function = symbolic_matrix_with_error[0:3,3]
    jacobian = position_function.jacobian(error_symbols)
    return jacobian

symbolic_jacobian = get_symbolic_jacobian()
calc_jacobian_for_angle = sym.lambdify([q_symbols,error_symbols], symbolic_jacobian, "numpy") 

print("symbolic calculations completed", time.strftime("%H:%M:%S", time.localtime()))

symbolic calculations started 19:38:04
symbolic calculations completed 19:38:18


In [3]:
#              General information: 
# We obtain "pose" from a kinematic model of a robot 
# pose = f(q,error) where q - is 6x1 vector of robot links angles (aka robot configuration)
# error = [d0, a0, θ0, α0, d1,a1 .... d5,a5,θ5,α5] - 24 error parameters 
# pose contains information (x,y,z) of object of our interest (and also rotation component which we ignore) 

# draft function which generates n evenly distributed q vectors 
# rewrite it for your purpose
def get_uniform_joints_angles_within_limits(number_of_poses):
    # angle limits for each robot joint in deg
    q_min = np.array ([-20,-50,-85,-145,-105,-195], dtype = 'float')
    q_max = np.array([95,55,40,145,105,195], dtype = 'float')
    # tranform angles to radians
    angles_to_rad(q_min)
    angles_to_rad(q_max)
    max_delta = q_max-q_min
    array_of_q = q_min.copy()
    q_current = q_min.copy()
    
    for i in range (number_of_poses-1):
        q_current += (1/number_of_poses) * max_delta
        array_of_q = np.vstack((array_of_q, q_current))
    return array_of_q

# draft function which generates kinematic error 
# rewrite it for your purpose
def generate_kinematics_error():
    real_error = np.zeros(24, dtype = 'float')
    for param in np.nditer(real_error, op_flags=['readwrite']):
            param[...] = param + np.random.normal(0,0.001)
    
    # probably these two parametrs below should be excluded from kinematic model
    # in case of problems restrict descent with the following coordinates 
    real_error [4] = 0.0
    real_error [8] = 0.0
    
    return real_error

# generate poses corresponding to a set of robot configurations (q) 
# return is presented as a column vector of (x,y,z) for each pose 
# dim(vector) = (N configurations * 3 , 1)
# vector = [x1,y1,z1, x2,y2.....xn,yn,zn]T
def get_poses_from_kinematic_model(set_of_q, kinematic_error):   
    poses_from_nominal_model = np.empty((0,1), float)
    for q in set_of_q:
        calculated_pose = np.array(get_pose_from_radians(q, kinematic_error)[0:3,3])
        calculated_pose.shape = (3,1)
        poses_from_nominal_model = np.vstack((poses_from_nominal_model,calculated_pose))
    return poses_from_nominal_model

# draft function which generates poses from hypotetical measuring device for each robot configuration   
# measured_pose = pose from real kinematic model + random noise corresponding to the device accuracy 
# enable_device_noise - bool variable that enables addition of a device noise
def get_data_from_measuring_device(set_of_q, enable_device_noise):
    real_error = generate_kinematics_error()
    poses_from_real_model = np.empty((0,1), float)
    
    for q in set_of_q:
        calculated_pose = np.array(get_pose_from_radians(q, real_error)[0:3,3])
        calculated_pose.shape = (3,1)
        poses_from_real_model = np.vstack((poses_from_real_model,calculated_pose))
    
    if (enable_device_noise):
        for pose in np.nditer(poses_from_real_model, op_flags=['readwrite']):
            pose[...] = pose + np.random.normal(0,0.005)      

    return poses_from_real_model

# builds jacobian for given robot configurations with giver error in kinematics
def form_full_jacobian(set_of_q,error):
    full_jacobian = np.empty((0,error.size), float)
    for q in set_of_q:
        full_jacobian = np.append(full_jacobian, calc_jacobian_for_angle(q,error), axis=0)
    return full_jacobian

# incompleted method as an example of how to use developed functions 
def least_squares_method ():
    
    number_of_configurations = 6
    robot_configurations = get_uniform_joints_angles_within_limits(number_of_configurations)
    
    # if kinematics_error equals to zero than kinematics corresponds to nominal
    kinematics_error = np.zeros(24, dtype = 'float')
  
    # vector is explained in function definition
    poses_from_model = get_poses_from_kinematic_model(robot_configurations,kinematics_error)
    print("nominal poses (x,y,z) merged in one column", poses_from_model.shape) 
    enable_noise_for_measuring_device = False
    poses_from_measuring_device = get_data_from_measuring_device(
        robot_configurations,enable_noise_for_measuring_device)
    print("measured poses (x,y,z) merged in one column",poses_from_measuring_device.shape)        
    # column vector of residuals for all robot configurations [dx0,dy0,dz0,dx1,dy2,.... ]T
    error_vector = poses_from_model - poses_from_measuring_device
    jacobian = form_full_jacobian(robot_configurations,kinematics_error)
    print(jacobian)
    print("shape of jacobian from all measurements", jacobian.shape)
    jt_by_j = jacobian.T @ jacobian
    print("jt_by_j is singular( ")   
    
print("numeric calculations started", time.strftime("%H:%M:%S", time.localtime()))
least_squares_method () 
print("numeric calculations completed", time.strftime("%H:%M:%S", time.localtime()))


numeric calculations started 19:38:56
nominal poses (x,y,z) merged in one column (18, 1)
measured poses (x,y,z) merged in one column (18, 1)
[[ 0.00000000e+00  9.39692621e-01  1.40946657e+02 -6.69437994e+01
   3.42020143e-01 -7.19846310e-01  1.83926577e+02  1.08903744e+02
  -3.42020143e-01  5.38985545e-01  2.93251414e+02  3.04202118e+02
  -7.69751131e-01 -6.37685806e-01 -2.79953397e+00 -7.50132867e-01
  -2.89829084e-02  9.08567729e-01  9.08567729e+01  2.89829084e+00
  -4.16730936e-01 -8.70107706e-01 -0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -3.42020143e-01  5.49235964e+02 -1.83926577e+02
   9.39692621e-01  2.62002630e-01 -6.69437994e+01  4.03334589e+02
  -9.39692621e-01 -1.96174695e-01 -1.06734786e+02  9.68481420e+02
   2.80166500e-01 -3.78288642e-01  8.52209614e+01  2.28348878e+01
   8.82272314e-01 -1.72711752e-01 -1.72711752e+01 -8.82272314e+01
  -4.37911195e-01 -6.15221356e-02  0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00  5.54032293e+01

In [25]:
#deprecated code to calculate pose for nominal kinematics and kinematics with error

def calculate_jacobian_at_angles(error_parameters,q_current):
    calc_jacobian_column()
    symbolic_matrix = get_symbolic_kinematics_matrix()    
    position_function = symbolic_matrix[0:3,3]
    jacobian = position_function.jacobian(error_symbols)
    calc_jacobian = sym.lambdify(error_symbols, jacobian, "numpy") 
    return calc_jacobian(*error_parameters)

def get_kinematics_model_with_noise(kinematic_parameters):
    for param in np.nditer(kinematic_parameters, op_flags=['readwrite']):
        param[...] = param + np.random.normal(0,0.15)
    return kinematic_parameters

# symbolic DH matrix
def get_symbolic_dh_matrix (d,a,θ,α):
    dh = sym.eye(4)
    dh[0,0] = sym.cos(θ)
    dh[0,1] = -sym.sin(θ) * sym.cos(α)
    dh[0,2] = sym.sin(θ) * sym.sin(α)
    dh[0,3] = a * sym.cos(θ)
    dh[1,0] = sym.sin(θ)
    dh[1,1] = sym.cos(θ)*sym.cos(α)
    dh[1,2] = -sym.cos(θ)* sym.sin(α)
    dh[1,3] = a *sym.sin(θ)
    dh[2,1] = sym.sin(α)
    dh[2,2] = sym.cos(α)
    dh[2,3] = d

    return dh

dh_matrix_without_error = get_symbolic_dh_matrix(d,a,θ,α)
calc_dh_matrix = sym.lambdify([d, a,θ,α], dh_matrix_without_error, "numpy") 

# calculate DH matrix with given parameters and angle θ
def get_matrix_for_angle (d,a,θ0,α, θ):
    matrix = calc_dh_matrix(d,a,np.deg2rad(θ0 +θ),np.deg2rad(α))
    return matrix
    
def get_pose (θ,kinematics):
    resulting_transform = sym.eye(4)
    for i in range(6):
        resulting_transform = resulting_transform @ get_matrix_for_angle(*kinematics[i], θ[i])
    return resulting_transform

q = np.array ([0,0,0,0,0,0], dtype = 'float')
nominal_pose = get_pose(q,get_nominal_DH_kinematic_parameters())

print("nominal pose",nominal_pose[0:3,3])

nominal pose Matrix([[1290.00000000000], [-1.33486501107061e-13], [1565.00000000000]])


In [10]:
a = np.array([1,2])
b = np.array([2,4])
for value in a:
    value = 2
print(a+b)

[3 6]
